In [1]:
import pandas as pd
import numpy as np
import gradio as gr
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

In [2]:
main_df = pd.read_csv('../3 - Cleaning & Transforming Data/2 cleaned_dataset_MERGED_DRUG(Mogran)_ALLELESEQ.csv')

In [3]:
allele_raw_df = pd.read_csv('../3 - Cleaning & Transforming Data/0 Selected_Protein_Seq_Dropped.csv')
seq_col = allele_raw_df.iloc[:,5:]
name_col = allele_raw_df['new_alleles']
allele_df = pd.concat([name_col, seq_col] ,axis=1)
allele_list = allele_df['new_alleles'].drop_duplicates().tolist()

In [4]:
drug_raw_df = pd.read_csv('../1 - Drugs Descriptors/drug_descriptors_dataset_morgan_dropped.csv')
drug_list = drug_raw_df['DRUG_NAME'].drop_duplicates().tolist()

In [5]:
model2 = torch.jit.load('../4 - Training & Testing/final/final').to('cuda')
model2.eval()

RecursiveScriptModule(
  original_name=Network
  (layers): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=ReLU)
    (2): RecursiveScriptModule(original_name=Dropout)
    (3): RecursiveScriptModule(original_name=Linear)
    (4): RecursiveScriptModule(original_name=ReLU)
    (5): RecursiveScriptModule(original_name=Dropout)
    (6): RecursiveScriptModule(original_name=Linear)
    (7): RecursiveScriptModule(original_name=ReLU)
    (8): RecursiveScriptModule(original_name=Dropout)
    (9): RecursiveScriptModule(original_name=Linear)
    (10): RecursiveScriptModule(original_name=ReLU)
    (11): RecursiveScriptModule(original_name=Dropout)
    (12): RecursiveScriptModule(original_name=Linear)
    (13): RecursiveScriptModule(original_name=Sigmoid)
  )
)

In [6]:
race_list =[
    "Caucasian (predominantly)",
    'Indian',
    'European',
    'Caucasian (North West)',
    'Caucasian (Spanish)',
    'Arab',
    'Han Chinese',
    'Japanese',
    'Korean',
    'Malay',
    'Thai',
    'Vietnamese',
    'Unknown'
]

In [7]:
race_dict ={
    "Caucasian (predominantly)":'Caucasian',
    'Indian':'Caucasian',
    'European':'Caucasian',
    'Caucasian (North West)':'Caucasian',
    'Caucasian (Spanish)':'Caucasian',
    'Arab':'Caucasian',
    'Han Chinese':'Mongol',
    'Japanese':'Mongol',
    'Korean':'Mongol',
    'Malay':'Mongol',
    'Thai':'Mongol',
    'Vietnamese':'Mongol',
    'Unknown':'Diverse'
}

In [8]:
def calculator(Race,Allele,Drug):

    Race = race_dict[Race]
    allele = allele_df[allele_df['new_alleles'] == Allele].copy()
    drug = drug_raw_df[drug_raw_df['DRUG_NAME'] == Drug].copy()
    dict_of_race = {
        "Black":[0],
        "Caucasian":[0],
        "Diverse":[0],
        "Mongol":[0],
    }
    dict_of_race[Race] = [1]
    race = pd.DataFrame(dict_of_race)

    allele.drop(columns=['new_alleles'], inplace=True)
    allele = allele.reset_index(drop=True)
    drug.drop(columns=['DRUG_NAME'], inplace=True)
    drug.reset_index(drop=True, inplace=True)
    input_df = pd.concat([race,drug,allele],axis=1)

    output = model2(torch.tensor(input_df.values, dtype=torch.float32).to('cuda'))
    output = output.detach().cpu().numpy()[0]
    return output

desc = "Predicting Type 2 ADRs with Alleles, Race, and Drugs (Morgan Fingerprints)\n Faculty of Pharmaceutical Science - Chulalongkorn University"

demo = gr.Interface(
    calculator,
    [gr.Dropdown(race_list), 
     gr.Dropdown(allele_list),
     gr.Radio(drug_list)], "number",
    live=True,
    title="Predicting Type 2 ADRs with Alleles, Race, and Drugs (Morgan Fingerprints)",
    allow_flagging='never',
    description = desc,
    article = "Click here for the Project's [Github](https://github.com/akumaharit/seniorproject)"
)
#demo.launch() 
demo.launch(share=True) #auth=("admin", "eieiza"), 

c:\Users\harit\AppData\Local\Programs\Python\Python37\lib\site-packages\gradio\blocks.py:256: UserWarning: api_name predict already exists, using predict_1
  "api_name {} already exists, using {}".format(api_name, api_name_)
c:\Users\harit\AppData\Local\Programs\Python\Python37\lib\site-packages\gradio\blocks.py:256: UserWarning: api_name predict already exists, using predict_2
  "api_name {} already exists, using {}".format(api_name, api_name_)


Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app


c:\Users\harit\AppData\Local\Programs\Python\Python37\lib\site-packages\torch\nn\modules\module.py:1194: UserWarning: FALLBACK path has been taken inside: torch::jit::fuser::cuda::runCudaFusionGroup. This is an indication that codegen Failed for some reason.
To debug try disable codegen fallback path via setting the env variable `export PYTORCH_NVFUSER_DISABLE=fallback`
 (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\jit\codegen\cuda\manager.cpp:336.)
  return forward_call(*input, **kwargs)
